<a href="https://colab.research.google.com/github/ylkhayat/cocolex/blob/main/example/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -L https://raw.githubusercontent.com/ylkhayat/cocolex/main/cocolex.py -o cocolex.py
!mkdir -p example
!curl -L https://raw.githubusercontent.com/ylkhayat/cocolex/main/example/dataloader_extras.py -o example/dataloader_extras.py
!curl -L https://raw.githubusercontent.com/ylkhayat/cocolex/main/example/dataloader.py -o example/dataloader.py
!touch example/__init__.py

In [ ]:
!pip install transformers==4.40.2 scikit-learn more-itertools faiss-cpu

In [ ]:
from example.dataloader import *
from example.dataloader_extras import *
from cocolex import CoCoLex

In [ ]:
cocolex_model = CoCoLex(model_name="mistralai/Mistral-7B-Instruct-v0.3", device=0, compile=False, use_flash_attention=False)

In [ ]:
import json

config = {
    "dataset_ratio": 1.0,
    "dataset": "oal_qa",
    "setup": "bm25_relevant_passages_oracle_documents",
    "split": "test",
    "top_k_passages": 3,
    "use_instructions": True,
}
preprocessor = ModelInputPreprocessor(
    config
)
work_dataset, _ = preprocessor.process_dataset(tokenizer=cocolex_model.tokenizer, max_tokens=cocolex_model.model.config.max_position_embeddings)

In [ ]:
new_record_processed = work_dataset[100]
print(json.dumps(new_record_processed, indent=4))

In [ ]:
import textwrap

contexts = [
    f"{new_record_processed['context_prefix']}\n\n{new_record_processed['context']}",
]

prompts = [
    new_record_processed["prompt"],
]

references = contexts.copy()

outputs = cocolex_model.generate(
                            prompts=prompts,
                            contexts=contexts,
                            references=references,
                            max_length=100,
                            )

In [ ]:
def print_block(title, content, width=160):
    print("=" * width)
    print(f"{title}:")
    print("-" * width)
    print("\n".join(textwrap.wrap(content, width=width)))
    print()

context = contexts[0]
prompt = prompts[0]
generation = cocolex_model.tokenizer.batch_decode(
    outputs, skip_special_tokens=True
)[0]

print_block("Contexts", context)
print_block("Prompts", prompt)
print_block("Generation", generation)